# 1. Basic Histogram

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [4]:
graph1 = tf.Graph()
with graph1.as_default():
    sess = tf.InteractiveSession()
    k = tf.placeholder(dtype=tf.float32)
    mean_moving_normal = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)
    summary1 = tf.summary.histogram("normal/moving_mean", mean_moving_normal)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/histogram_example")
    tf.global_variables_initializer().run()
    N = 10
    for i in range(N):
        k_val = i / float(N)
        summary_str, _ = sess.run([merged, mean_moving_normal], feed_dict={k: k_val})
        writer.add_summary(summary_str, global_step=i)

# 2. Multimodal Distributions

In [5]:
import tensorflow as tf

graph2 = tf.Graph()
with graph2.as_default():
    k = tf.placeholder(tf.float32)

    # Make a normal distribution, with a shifting mean
    mean_moving_normal = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)
    # Record that distribution into a histogram summary
    tf.summary.histogram("normal/moving_mean", mean_moving_normal)

    # Make a normal distribution with shrinking variance
    variance_shrinking_normal = tf.random_normal(shape=[1000], mean=0, stddev=1-(k))
    # Record that distribution too
    tf.summary.histogram("normal/shrinking_variance", variance_shrinking_normal)

    # Let's combine both of those distributions into one dataset
    normal_combined = tf.concat([mean_moving_normal, variance_shrinking_normal], 0)
    # We add another histogram summary to record the combined distribution
    tf.summary.histogram("normal/bimodal", normal_combined)

    # Add a gamma distribution
    gamma = tf.random_gamma(shape=[1000], alpha=k)
    tf.summary.histogram("gamma", gamma)

    # And a poisson distribution
    poisson = tf.random_poisson(shape=[1000], lam=k)
    tf.summary.histogram("poisson", poisson)

    # And a uniform distribution
    uniform = tf.random_uniform(shape=[1000], maxval=k*10)
    tf.summary.histogram("uniform", uniform)

    # Finally, combine everything together!
    all_distributions = [mean_moving_normal, variance_shrinking_normal,
                         gamma, poisson, uniform]
    all_combined = tf.concat(all_distributions, 0)
    tf.summary.histogram("all_combined", all_combined)

    summaries = tf.summary.merge_all()

    # Setup a session and summary writer
    sess = tf.Session()
    writer = tf.summary.FileWriter("./logs/histogram_example")

    # Setup a loop and write the summaries to disk
    N = 400
    for step in range(N):
      k_val = step/float(N)
      summ = sess.run(summaries, feed_dict={k: k_val})
      writer.add_summary(summ, global_step=step)